In [21]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

In [22]:
from graphai import node, router, Graph
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_text_splitters import CharacterTextSplitter
from pydantic import BaseModel, Field
import asyncio

In [23]:
from prompts.prompts import language_translation_system_prompt


In [24]:
class State(BaseModel):
    languages: list[str] = Field(default_factory=list)
    original_text: str = ""
    translations: dict = Field(default_factory=dict)
    character_split: str = "\n"
    chunk_size: int = 2000


In [25]:
llm_fast = ChatOpenAI(model="gpt-4o-mini")

language_translation_prompt = ChatPromptTemplate([
    ('system', language_translation_system_prompt),
    ('human', '{input}')
])
chain = language_translation_prompt | llm_fast | StrOutputParser()


In [26]:
@node(start=True)
async def node_start(languages: list, original_text: str, character_split: str = "\n"):
    """Start node that initializes processing"""
    print('a')
    return {
        "languages": languages,
        "original_text": original_text,
        "character_split": character_split
    }


@router
async def translation_router(languages: list, original_text: str, character_split: str):
    """Router that prepares chunks for translation"""
    print('b')
    text_splitter = CharacterTextSplitter(
            separator=character_split,
            chunk_size=5,
            chunk_overlap=0,
            length_function=len,
            is_separator_regex=False
        )
    
    documents = text_splitter.create_documents([original_text])
    # Return choice and state
    route = {
        "choice": "translate_chunk",
        "chunks": [
            {
                "language": lang,
                "text": page.page_content,
                "character_split": character_split
            }
            for lang in languages
            for page in documents
        ]
    }

    print("router is returning: ", route)

    # Route to translate_chunk with all necessary data
    return route 

@node(stream=True)
async def translate_chunk(chunks: list, callback = None):
    """Translate chunks in parallel"""
    async def process_chunk(chunk):
        output = await chain.ainvoke({
            "language": chunk["language"],
            "input": chunk["text"]
        })
        if callback:
            await callback.acall(f"Translated chunk for {chunk['language']}")
        return (chunk["language"], output + chunk["character_split"])

    # Process all chunks in parallel
    results = await asyncio.gather(*[process_chunk(chunk) for chunk in chunks])
    
    # Combine results
    translations = {}
    for language, translation in results:
        if language not in translations:
            translations[language] = ""
        translations[language] += translation

    return {"translations": translations}


@node(end=True)
async def node_end(translations: dict):
    """Final node that returns translations"""
    return {"output": translations}


In [27]:
text = """
The Metamorphosis might not "click" unless you've experienced the kind of dissociation from reality Kafka is describing. 
It also requires you to read the story abstractly—it’s less about what happens and more about how it happens, or why it happens. 
If you’re looking for a straightforward story, Kafka’s stuff will probably frustrate you, because the disorienting, anxious effect 
is intentional.

Kafka’s work explores alienation, anxiety, and seeking meaning in a universe that seemingly lacks any. 
There are a bunch of ways to pick through those themes: What is alienation? Is it personal, social, moral, practical, etc? 
That is, is Gregor a “monstrous vermin/bug” because of anything he did—or because of how he thinks about himself—or how his 
family thinks about and treats him—or how society treats him—or for genuine practical reasons, like the way his “metamorphosis” 
impairs his body and physical abilities? The metamorphosis is a source of anxiety—but is it really? Is it the actual change itself 
that causes Gregor anxiety, or is it the expectations he places on himself (e.g. the clash of trying to do his old normal job as 
if he still had his old normal body), or is it the expectations his family/society places on him? Is the metamorphosis really as 
meaningless and random as it seems? Or is it a reflection of something deeper? After all, his family seems mildly shocked at first, 
but all things considered they go back to treating him roughly the same as before. One of the messages you could take from the Metamorphosis 
is that every “random/meaningless shock” in your conception of reality is actually a wake up call, a reflection of a deeper reality you have 
been able to ignore so far by keeping busy with the surface practicalities. Or you could read it as a black comedy. Or an absurd tragedy. 
Or etc. It might be easier to focus less on what the story says, and more on how you read it, and why you read it that way, and what 
that can tell you about yourself.
"""

In [28]:
graph = Graph()

for node_fn in [node_start, translate_chunk, node_end]:
    graph.add_node(node_fn)

graph.add_router(
    sources=[node_start],
    router=translation_router,
    destinations=[translate_chunk]
)

graph.add_edge(source=node_start, destination=translation_router)
graph.add_edge(source=translate_chunk, destination=node_end)

graph.compile()
#graph.visualize()


initial_state = {
    "languages": ["Albanian", "Italian", "French"],
    "original_text": text,
    "character_split": "."
}

callback = graph.get_callback()
response = asyncio.create_task(graph.execute(initial_state))

print("Streaming tokens:")
async for token in callback.aiter():
    print(f"Token received: {token}")

final_result = await response
print("\nFinal result:", final_result)

2025-01-05 14:57:31 - langchain_text_splitters.base - WARNING - base.py:122 - _merge_splits() - Created a chunk of size 120, which is longer than the specified 5
2025-01-05 14:57:31 - langchain_text_splitters.base - WARNING - base.py:122 - _merge_splits() - Created a chunk of size 129, which is longer than the specified 5
2025-01-05 14:57:31 - langchain_text_splitters.base - WARNING - base.py:122 - _merge_splits() - Created a chunk of size 148, which is longer than the specified 5
2025-01-05 14:57:31 - langchain_text_splitters.base - WARNING - base.py:122 - _merge_splits() - Created a chunk of size 103, which is longer than the specified 5
2025-01-05 14:57:31 - langchain_text_splitters.base - WARNING - base.py:122 - _merge_splits() - Created a chunk of size 589, which is longer than the specified 5
2025-01-05 14:57:31 - langchain_text_splitters.base - WARNING - base.py:122 - _merge_splits() - Created a chunk of size 394, which is longer than the specified 5
2025-01-05 14:57:31 - langch

Streaming tokens:
a
b
router is returning:  {'choice': 'translate_chunk', 'chunks': [{'language': 'Albanian', 'text': 'The Metamorphosis might not "click" unless you\'ve experienced the kind of dissociation from reality Kafka is describing', 'character_split': '.'}, {'language': 'Albanian', 'text': 'It also requires you to read the story abstractly—it’s less about what happens and more about how it happens, or why it happens', 'character_split': '.'}, {'language': 'Albanian', 'text': 'If you’re looking for a straightforward story, Kafka’s stuff will probably frustrate you, because the disorienting, anxious effect \nis intentional', 'character_split': '.'}, {'language': 'Albanian', 'text': 'Kafka’s work explores alienation, anxiety, and seeking meaning in a universe that seemingly lacks any', 'character_split': '.'}, {'language': 'Albanian', 'text': 'There are a bunch of ways to pick through those themes: What is alienation? Is it personal, social, moral, practical, etc? \nThat is, is G

2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Token received: Translated chunk for Albanian
Token received: Translated chunk for Albanian
Token received: Translated chunk for Albanian
Token received: Translated chunk for Albanian


2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:25 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Token received: Translated chunk for French


2025-01-05 14:58:28 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Token received: Translated chunk for Italian


2025-01-05 14:58:30 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-05 14:58:30 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Token received: Translated chunk for French
Token received: Translated chunk for French
Token received: <graphai:end:translate_chunk>
Token received: <graphai:END>

Final result: {'languages': ['Albanian', 'Italian', 'French'], 'original_text': '\nThe Metamorphosis might not "click" unless you\'ve experienced the kind of dissociation from reality Kafka is describing. \nIt also requires you to read the story abstractly—it’s less about what happens and more about how it happens, or why it happens. \nIf you’re looking for a straightforward story, Kafka’s stuff will probably frustrate you, because the disorienting, anxious effect \nis intentional.\n\nKafka’s work explores alienation, anxiety, and seeking meaning in a universe that seemingly lacks any. \nThere are a bunch of ways to pick through those themes: What is alienation? Is it personal, social, moral, practical, etc? \nThat is, is Gregor a “monstrous vermin/bug” because of anything he did—or because of how he thinks about himself—or

In [29]:
final_result['output']

{'Albanian': 'Metamorfoza ndoshta nuk do të "kapet" nëse nuk ke përjetuar llojin e shkëputjes nga realiteti që Kafka përshkruan..Kjo gjithashtu kërkon që të lexosh historinë në mënyrë abstrakte - nuk është kaq shumë për atë që ndodh, por më shumë për mënyrën se si ndodh, ose përse ndodh..Nëse po kërkoni një histori të thjeshtë, punimet e Kafkës ndoshta do t\'ju frustronin, sepse efekti dezorientues dhe ankthioz është qëllimshëm..Vepra e Kafka-s shqyrton ndjenjën e alienimit, ankthit dhe kërkimin e kuptimit në një univers që duket se nuk ka asgjë..Ka shumë mënyra për të shqyrtuar këto tema: Çfarë është alienimi? A është ai personal, social, moral, praktik, etj? \nDo thotë, a është Gregori një "krimb monstruoz" për shkak të asaj që ai bëri—apo për shkak të mënyrës si mendon për veten—apo si e sheh dhe e trajton familja e tij—apo si e trajton shoqëria—apo për arsye të vërteta praktike, siç është mënyra se si "metamorfoza" e tij dëmton trupin dhe aftësitë fizike? Metamorfoza është një buri